In [5]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm

In [2]:
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [6]:
names = pd.concat(places_names, axis=0)

In [10]:
names

0         元记电气焊补胎钣金
1            元汇津通卸载
2           顺福轮胎修理部
3        宏顺汽车维修保养中心
4        浩博连接起来汽车维修
            ...    
15004    扶梯(友谊汉友百货)
15005    扶梯(友谊汉友百货)
15006    楼梯(友谊汉友百货)
15007    客梯(友谊汉友百货)
15008    楼梯(友谊汉友百货)
Name: name, Length: 758397, dtype: object

In [11]:
sf = shapefile.Reader('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/天津市_公共设施_202209.shp')

In [ ]:
for s in sf:
    print(s.record[0])